<a href="https://colab.research.google.com/github/andrav15/Regresor1/blob/main/Regresor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import math

In [ ]:
#crearea datelor
x = torch.linspace(-math.pi, math.pi, 2000)#tensor care contine 2000 de valori ale lui x uniform distribuite (-pi;pi)
y = torch.sin(x)#tensor y=sinusul val din x->iesirea


In [ ]:
#pregatirea intrarilor
p = torch.tensor([1, 2, 3])#reprezinta puterile lui x
xx = x.unsqueeze(-1).pow(p)#contine puterile lui x(x^1,x*2,x^3)
#x.unsqueeze(-1):(200,)->(2000,1) ad undei dim supl
#cand ridicam x.unsqueeze(-1) la puteri(dim p =3)->tensor(2000,3) adica col1:x^1,col2:x^2,col3:x^3



In [ ]:
#definim modelul
model = torch.nn.Sequential(#model neural in care straturile sunt aplicate secvential
    torch.nn.Linear(3, 1),#strat liniar:calc iesirea fol o combinatie liniara a intrarilor
                            #in dim 3, iesirea dim 1
    torch.nn.Flatten(0, 1)#strat de aplatizare:transf iesirea care poate avea mai multe dim intr-un tensor 1D
                            #aici iesirea are deja dim1->ajusteaza dim pt a se potrivi cu dim dorita a lui y
)
#in acest caz invata o rel de forma y=bias+w1x+w2x^2+w3x^3, unde bias si w1,w2,w3 sunt optimizate in timpul antrenarii pt a aprox cat mai bn y=sin(x)

In [ ]:
#fct de pierdere: eroarea patratica medie(MSE)
#MSE este preferata in regresie pt ca penalizeaza erorile mari ceea ce este esential pt prezicerea val numerice continue
loss_fn = torch.nn.MSELoss(reduction='sum')
#argumentul reduction='sum': indica ca val calculate pt erorile patratice sunt sumate la final
#alegem sum: pierderea creste prop cu dim lotului
#daca alegeam mean: pierderea era independenta de dim lotului

In [ ]:
learning_rate = 1e-6#val f mica pt o antrenare stabila  si evita suprainvatarea

In [ ]:
#antrenarea astfel incat sa minimizam erorile
for t in range(2000):#2000 de epoci
    y_pred = model(xx)#pt fiecare epoca calc predicatia
    loss = loss_fn(y_pred, y)#evalueaza pierderea:compara predictiile y_pred cu val reale y si calc fol MSE
    if t % 100 == 99:
      print(t,loss.item())#afiseaza pierderea la fiecare 100 de epoci pt a urmari progresul antrenarii
    model.zero_grad()#resetarea gradientului pt a nu se acumula toate val grad care ar duce la erori mari
    loss.backward()#actualizeaza greutatile prin backpropagation->partea cheie a invatarii
    with torch.no_grad():#nu dorim sa urmarim grad; pytorch tine evidenta grad pt calcularea backpropgation dar in timpul actualizarii nu avem nev
        for param in model.parameters():#iteram prin toti param(bias,w)
            param -= learning_rate * param.grad#ajusteaza fiecare param
            #param.grad- grad pierderii in raport cu acel param calculat mai sus
            #daca learning_rate este mic->ajustarea va fi mica;lr mare->ajustare mare a param

99 518.34033203125
199 351.58795166015625
299 239.61337280273438
399 164.36012268066406
499 113.74287414550781
599 79.6661148071289
699 56.70390319824219
799 41.21687316894531
899 30.761289596557617
999 23.69558334350586
1099 18.915843963623047
1199 15.679121017456055
1299 13.485005378723145
1399 11.996028900146484
1499 10.98445987701416
1599 10.296465873718262
1699 9.828039169311523
1799 9.508742332458496
1899 9.290839195251465
1999 9.141968727111816


In [ ]:
linear_layer = model[0]#extragem primul strat care e un strat liniar pt accesarea directa a param sai(w si bias)


In [ ]:
print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')
#afisarea modelului fial ca o ecuatie a unei fct polinomiale de gradul 3

Result: y = -0.014068412594497204 + 0.8449088931083679 x + 0.0024270338471978903 x^2 + -0.09164739400148392 x^3
